# Classes

Classes provide a means of bunding data and functionality together. Creating a new class creates a new type of object, allowing new instances of that type to be made. Each class instance can habr attributes attached to it for maintainig its state. Class instance can also have methods for modifying its state.

Objects have individuality, and multiple names can be bound to the same object.

### Python Scopes and Namespaces

Class definitionas play some neat tricks with namespaces, and you need to know how scopes and namespaces work to fully understand what's going on. Incidentally, knowledge about this subject is useful for any advanced Python programmer.

A **_namespace_** is a mapping from names to objects. Most namespaces are currently implemented as Python dictionaries, but that's normally not noticeable in any way, and it may change in the future. Examples of namespaces are: the set of built-in names; the global names in a module; and the local names in a function invocation.

The important thing to know about namespaces is that there is absolutely no relation between names in different namespaces; for instance, two different modules may both define a function `maximize` without confusion - users of the module must prefix it with the module name.

The word **_attribute_** is used for any name following a dot - for example, in the expression `z.real`, `real` is an attribute of the object `z`. In the expression `modname.funcname`, `modname` is a module object and `funcname` is an attribute of it.

Attributes may be read-only or writable. In the latter case, assignment to attributes is possible. Module attribute are writable: you can write `modname.the_answer = 42`. Writable attributes may also be deleted with `del` statement.

Namespaces are created at different moments and have different lifetimes. The namespace containing the built-in names is created when the Python interpreter starts up, and is never deleted. The global namespace for a module is created when the module definition is read in; normally, module namespaces also last until the interpreter quits. The statements executed by the top-level invocation of the interpreter, either read from a script file or interactively, are considered part of a module called `__main__`, so they have their own global namespace.

The local namespace for a function is created when the fucntion is called, and deleted when the function returns or raised exception that is not handled within the function.

A **_scope_** is a textual region of a Python program where a namespace is directly accessible. "Directly accessible" here means that an unqualified to a name attempts to find the name in the namespace.

Although scopes are determined statically, they are used dynamically. At any time during execution, ther are 3 or 4 nested scopes whose namespaces are directly accessible:

* the innermost scope, which is searched first, contains the local names
* the scopes of any enclosing functions, which are searched starting with the nearest enclosing scope, contains non-local, but also non-global names
* the next-to-last scope contains the current module's global names
* the outermost scope is the namespace containing built-in names

I a name is declared global, then all references and assignments go directly to the middle scope containing the module's global names.

Usually, the local scope references the local names of the (textually) current function. Outside functions, the local scope references the same namespace as the global scope: the module's namespace. Class definitions place yet another namespace in the local scope.

It is important to realize that scopes are determined textually: the global scope of function defined in a module is that module's namespace, no matter from where or by what alias tha function is called. On the other hand, the actual search for names is done dynamically, at run time.

The `global` statement can be used to indicate that particular variables live in the global scope and should be rebound there; the `nonglobal` statement indicates that particular variables live in a enclosing scope and should be rebound there.

In [3]:
def scope_test():
    def do_local():
        spam = "local spam"
        
    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"
    
    def do_global():
        global spam
        spam = "global spam"
    
    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


Note how the local assignment (which is default) didn’t change scope_test’s binding of spam. The nonlocal assignment changed scope_test’s binding of spam, and the global assignment changed the module-level binding.

You can also see that there was no previous binding for spam before the global assignment.

## A First Look at Classes

### Class Definition Syntax

The simplpest form of class definition looks like this:  
`class ClassName:
    <statement-1>
    .            
    .            
    .            
    <statement-N>`
    
Class definitions, like function definitons (`def` statements) must be executed before they have any effect.  
In practice, the statements inside a class definition will usually bef unction definitions, but other statements are allowed, and sometimes useful. The function definitions inside a class normally have a peculiar form of argument list, dictated by the calling conventions for methods.  
When a class definition is entered, a new namespace is created, and used as the local scope - thus, all assignments to local variables go into this new namespace.

### Class Objects

Class objects supports two kinds of operations: attribute references and instantiation.

_Attribute reference_ use the standard syntax used for all attribute referernces in Python: `obj.name`. Valid attribute names are all the names that were in the class's namespace when the class object was created. So, if the class definition looked like this:

In [4]:
class MyClass:
    """A simple example class"""
    i = 12345
    
    def f(self):
        return 'hello world'

then `MyClass.i` and `MyClass.f` are valid attribute references, returning an integer and a function object, respectively. Class attribute can also be assigned to, so you can change the value of `MyClass.i` by assignmnet. `__doc__` is also a valid attribute, returning the docstring belonging to the class: `"A simple example class"` 

In [5]:
MyClass.i

12345

In [8]:
MyClass.__doc__

'A simple example class'

Class _instantiation_ uses function notation. Just pretrnd that the class object is a parameterless function that returns a new instance of the class.

In [17]:
x = MyClass()

creates a new instance of the class and assigns this object to the local variable x.

The instantitation operation ("calling" a class object creates an empty object. Many classes like to create objects with instances customized to a specific initial state. Therfore a class may define a special method named `__init__()`, like this:

In [18]:
def __init__(self):
    self.data = []

When a class defines an `__init__()` method, class instantiation automatically invokes `__init__()` for the newly-created class instance. So in this example, a new, inititalized instance can be obtained by:

In [19]:
x = MyClass()

In [20]:
class Complex:
    def __init__(self, realpart, imagpart):
        self.r = realpart
        self.i = imagpart

In [21]:
x = Complex(3.0, -4.5)

In [22]:
x.r, x.i

(3.0, -4.5)

### Instance Objects

The only operations understood by instance objects are attribute references. There are two kinds of valid attribute names: data attributes and methods.

_Data attributes_ need not be declared; like local variables, they spring into existence when they are first assigned to. For example, if `x` is the instance of `MyClass` created above, the following piece of code will print the value 16, without leaving a trace:

In [23]:
x.counter = 1
while x.counter < 10:
    x.counter = x.counter * 2
print(x.counter)
del x.counter

16


The other kind of instance attribute reference is a _method_. A method is a function that "belongs to" an object.

In Python, the term method is not unique to class instance: other object types can have methods as well. For example, list objects have method called append, insert, remove, sort and so on.

By definition, all attributes of a class that are function objects define corresponding methods of its instances. So in our example, `x.f` is a valid method reference, since `MyClass.f` is a function, but `x.i` is not, since `MyClass.i` is not. But `x.f` is not the sam thing as `MyClass.f` - it is a method object, not function object.

### Method Objects

Usually, a method is called right after it is bound:

In [25]:
x = MyClass()
x.f()

'hello world'

However, it is not necessary to call a method right away: `x.f` is a method object, and can be stored away and called at a later time. For example:

In [26]:
xf = x.f
while True:
    print(xf())

hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hell

hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hell

hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hell

hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hell

hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hell

...

hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hell

hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hell

hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hell

hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hell

KeyboardInterrupt: 

will continue to print `hello world` until the end of time.

What exactly happens when a method is called? You may have noticed that `x.f()` was called without an argument above, even though the function definition for f() specified an argument.

What happened to the argument? Surely Python raises an exception when a function that requires an argument is called without any - even if the argument isn't actually used...

In out example, the call `x.f()` is exactly equivalent to `MyClass.f(x)`. In general, calling a method with a list of n arguments is equivalent to calling the corresponding function with an argument list that is created by inserting the method's instance object before the first argument.

### Class and Instance Variables

Generally speaking, instance variables are for data unique to each instance and class variables are for attributes and methods shared by all instances of the class:

In [27]:
class Dog:
    kind = 'canine'  # class variable shared by all instances
    
    def __init__(self, name):
        self.name = name # instance variable unique to each instance

In [28]:
d = Dog('Fido')
e = Dog('Buddy')

In [29]:
d.kind # shared by all dogs 

'canine'

In [30]:
e.kind # shared by all dogs

'canine'

In [33]:
e.name # unique to e

'Buddy'

In [32]:
d.name # unique to d

'Fido'

Shared data can have possibly surprising effects with involving mutable objects such as lists and dictionaries. For example, the tricks list in the following code should not be used as a class variable because just a single list would be shared by all Dog instances:

In [34]:
class Dog:
    tricks = [] # mistaken use of a class variable
    
    def __init__(self, name):
        self.name = name
        
    def add_trick(self, trick):
        self.tricks.append(trick)

In [35]:
d = Dog('Fido')
e = Dog('Buddy')

In [36]:
d.add_trick('roll over')
e.add_trick('play dead')

In [37]:
d.tricks

['roll over', 'play dead']

Correct design of the class should use an instance variable instead:

In [38]:
class Dog:
    
    def __init__(self, name):
        self.name = name
        self.tricks = [] # creates a new empty list for each dog
        
    def add_trick(self, trick):
        self.tricks.append(trick)

In [39]:
d = Dog('Fido')
e = Dog('Buddy')

In [40]:
d.add_trick('roll over')
e.add_trick('play dead')

In [41]:
d.tricks

['roll over']

In [42]:
e.tricks

['play dead']

## Random Remarks

I the same attribute name occurs in both an instance and in a class, then attribute lookup priorities the instance:

In [43]:
class Warehouse:
    purpose = 'storage'
    region = 'west'

In [44]:
w1 = Warehouse()

In [45]:
print(w1.purpose, w1.region)

storage west


In [46]:
w2 = Warehouse()

In [47]:
w2.region = 'east'
print(w2.purpose, w2.region)

storage east


Data attributes may be referenced by methods as well as by ordinary users of an object. In other words, classes are not usable to implement pure abstract data types.

There is no shorthand for referencing data attributes from within methods. There is no chance of confusing local variables and instance variables when glancing through a method.

often, the first argument of a method is called `self`. This nothing more than a convention: the name `self` has absolutely no special meaning to Python.

Any function object that is a class attribute defines a method for instances of that class. It is not necessary that the function definition is textually enclosed in the class definition: assigning a function object to a local variable in the class is also ok. For example:

In [48]:
# Function defined outside the class
def f1(self, x, y):
    return min(x, x+y)

class C:
    f = f1
    
    def g(self):
        return 'hello world'
    
    h = g

Now `f`, `g` and `h1` are all attributes of class C that refers to function objects, and consequently they are all methods of instances of C - `h` being exactly equivalent to `g`. Note that this practice usually only serves to confuse the reader of a program.

Methods may call other methods by using method attributes of the `self` argument:

In [49]:
class Bag:
    def __init__(self):
        self.data = []
        
    def add(self, x):
        self.data.append(x)
        
    def addtwice(self, x):
        self.add(x)
        self.add(x)

In [55]:
b = Bag()

In [57]:
b.add(2)

In [58]:
b.addtwice(2)

In [60]:
b.data

[2, 2, 2]

Methods may reference global names in the same way as ordinary functions. The global scope associated with a method is the module containing its definition. While one rarely encounters a good reason for using global data in a method, there are many legitimate uses of the global scope: for one thing, functions and modules imported into the global scope can be used by methods, as well as functions and classes defined in it. Usually, the class containing the method is itself defined in this global scope, and in the next section we’ll find some good reasons why a method would want to reference its own class.

## Inheritance

The syntax for a derived class definition looks like this:

`class DerivedClassName(BaseClassName):
    <statement-1>
    .            
    .            
    .            
    <statement-N>`

The name BaseClassName must be defined in a scope containing the derived class definition. In place of a base class name, other arbitrary ecpressions are also allowed. This can be useful, for example, when the base class is defined in another module:

`class DerivedClassName(modname.BaseClassName):`

Execution of a  derived class definition proceeds the same as for a base class. When the class object id constructed, the base class is remembered.

There is nothing special about instantiation of derived classes: `DerivedClassName()` creates a new instance of the class. Method references are resolved as follows: the corresponding class attributes is searched, descending down the chain of base classes if necessary, and the method reference is valid of this yields a function object.

Derived classes may override methods of their base classes. Because methods have no special privileges when calling other methods of the same object, a method of a base class that calls another method defined in the same base class may end up calling method of a derived class that overrides it.

An overriding method in a derived class may in fact want to extend rather than simply replace the base class method of the same name. There is a simple way to call the base class method directly: just call `BaseClassName.methodname(self, arguments)`.

Python has two built-in functions that work with inheritance:
* Use `isinstance()` to check an instance's type: `isinstance(obj, int)` will be `True` only if `obj.__class__` is int or some class derived from int.
* Use `issubclass()` to check class inheritance:
`issubclass(bool, int)` is `True` since bool is a subclass if int. However, `issubclass(float, int)` is `False` since float is not a subclass of int.

### Multiple Inheritance

Python supports a form of multiple inheritance as well. A class definition with multiple base classes look loke this:

`class DerivedClassName(Base1, Base2, Base3):
    <statement-1>
    .............
    .............
    .............
    <statement-N>`

For most purposes, in the simplest cases, you can think of the search for attributes inherited from a parent class as depth-first, left-to-right, not searching twice in the same class where there is an overlap in the hierarchy. Thus, if an attribute is not found in DerivedClassName, it is searched for in Base1, then (recursively) in the base classes of Base1, and if it was not found there, it was searched for in Base2, and so on.

Dynamic ordering is necessary because all cases of multiple inheritance exhibit one or more diamond relationships (where at least one of the parent classes can be accessed through multiple paths from the bottommost class). For example, all classes inherit from object, so any case of multiple inheritance provides more than one path to reach object. To keep the base classes from being accessed more than once, the dynamic algorithm linearizes the search order in a way that preserves the left-to-right ordering specified in each class, that calls each parent only once, and that is monotonic (meaning that a class can be subclassed without affecting the precedence order of its parents). Taken together, these properties make it possible to design reliable and extensible classes with multiple inheritance.

## Private Variables

"Private" instance variables that cannot be accessed except from inside an object don't exist in Python. However, there is a convention that is followed by most Python code: a name prefixed with an underscore (e.g. `_spam`) should be treated as a non-public part of the API (whether it is a function, a method or a data member).

Since there is a valid use-case for class-private members, there is limited support for such mechanism, called _name mangling_. Any identifier of form `__spam` (at least two leading underscores, at most one trailing underscore) is textually replace with `_classname__spam`, where `classname` is the current class name with leading underscore(s) stripped. This mangling is done without regarfd to the suntactic position of the identifier, as long as it occurs within the definition of class.

In [61]:
class Mapping:
    def __init__(self, iterable):
        self.items_list = []
        self.__update(iterable)
        
    def update(self, iterable):
        for item in iterable:
            self.items_list.append(item)
    
    __update = update
    
class MappingSubclass(Mapping):
    def update(self, keys, values):
        # provides new signature for update()
        # but does not break __init__()
        for item in zip(keys, values):
            self.item_list.append(item)

The above example would work even if `MappingSubclass` are to introduce a `__update` identifier since it is replaced with `_Mapping__update` in the Mapping class and `_MappingSubclass__update` in the MappingSubclass class respectively.

Note that the mangling rules are designed mostly to avoid accidents; it is possible to access or modify a variable that is considered private. This can even be useful in special circumstances, such as in the debugger.

## Iterators

The use of iterators pervades and unifies Python. Behind the scenes, the for statement calls `iter()` on the container object. The function returns an iterator object that defines the method `__next__()` which accesses elements in the container one at a time. When there are no more elements, `__next__()` raises a `StopIteration` exception which tells the for loop tp terminate. You can call the `__next__()` method using the `next()` built-in functionl this exampple shows how it works:

In [62]:
s = 'abc'
it = iter(s)

In [63]:
it

In [64]:
next(it)

'a'

In [65]:
next(it)

'b'

In [66]:
next(it)

'c'

In [67]:
next(it)

StopIteration: 

Having seen the mechanics behind the iterator protocol, it is easy to add iterator behavior to your classes. Define an `__iter__()` method which returns an object with a `__next__()` method. If the class defines `__next__()`, then `__iter__()` can just return `self`:

In [68]:
class Reverse:
    """Iterstor for looping over a sequence backwords"""
    def __init__(self, data):
        self.data = data
        self.index = len(data)
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]

In [77]:
rev = Reverse('spam')

In [78]:
iter(rev)

In [79]:
for char in rev:
    print(char)

m
a
p
s


## Generators

Generators are a simple and powerful tool for creating iterators. They are written like regular functions but use the `yield` statement whenever they want to return data. Each time `next()` is called on it, the generator resumes where it left off (it remembers all the data values and which statement was last executed). An example shows that generators canb e trivially easy to create:

In [80]:
def reverse(data):
    for index in range(len(data)-1, -1, -1):
        yield data[index]

In [81]:
for char in reverse('golf'):
    print(char)

f
l
o
g


What makes generators so compact is that the `__iter__()` and `__next__()` methods are created automatically.

Another key feature is that the local variables and execution state are automatically saved between calls. This made the function easier to write and much more clear than an approach using instance variables like `self.index` and `self.data`.

In addition to automatic method creation and saving program state, when generators terminate, they automatically raise `StopIteration`.

## Generator Expressions

Some simple generators can be coded succinctly as expressions using a syntax similar to list comprehensions but with parentheses instead of square brackets. These expressions are designed for situations where the generator is used right away by an enclosing function. Generator expressions are more compact but less versatile than full generator definitions and tend to be more memory friendly than equivalent list comprehensions.

In [82]:
sum(i*i for i in range(10))

285

In [83]:
xvec = [10, 20, 30]
yvec = [7, 5, 3]

In [84]:
sum(x*y for x,y in zip(xvec, yvec))

260

In [88]:
data = 'golf'

In [89]:
list(data[i] for i in range(len(data)-1, -1, -1))

['f', 'l', 'o', 'g']